In [ ]:
import string
import sys
import getopt
import re

In [ ]:
print("Hello")

Hello


#Cipher Code Essentials

### Basic Operations

#### Format conversion

In [ ]:
# converts integer to bitstring having minimum length = minlen. 
# If length of obtained string < milen, 0s appended at the end
def bitstring(n, minlen=1):
    if minlen < 1:
        print("ValueError: a bitstring must have at least 1 char")
    if int(n) < 0:
        print("ValueError: bitstring representation undefined for neg numbers")
    result = str(bin(int(n)))[2:]
    '''while n > 0:
        if n & 1:
            result = result + "1"
        else:
            result = result + "0"
        n = n >> 1'''
    if len(result) < minlen:
        result = "0" * (minlen - len(result)) + result 
    return result


# reversing a string
def reverseString(s):
    newstr = ""
    l = list(s)
    l.reverse()
    for i in l:
      newstr += i
    return newstr


# splitting 128 bit string into 4 32 bit strings for bitslice format
def quadSplit(b128):
    if len(b128) != 128:
        print("ValueError: must be 128 bits long, not " + len(b128))
    result = []
    for i in range(4):
        result.append(b128[(i*32):(i+1)*32])
    return result

# concatinating 4 32 bit words into 128 bit string
def quadJoin(l4x32):
    if len(l4x32) != 4:
        print("ValueError: need a list of 4 bitstrings, not " + len(l4x32))
    return l4x32[0] + l4x32[1] + l4x32[2] + l4x32[3]

In [ ]:
bitstring(1, 4)

'0001'

#### XOR

In [ ]:
# XOR of two binary strings of equal length
def binaryXor(n1, n2):
    if len(n1) != len(n2):
        print("ValueError: can't xor bitstrings of different lengths ({} and {})".format(len(n1), len(n2)))
    result = ""
    for i in range(len(n1)):
        if n1[i] == n2[i]:
            result = result + "0"
        else:
            result = result + "1"
    return result

# XOR of arbitrary number of bitstrings of equal length
def xor(*args):
    if args == []:
        print("ValueError: at least one argument needed")
    result = args[0]
    for arg in args[1:]:
        result = binaryXor(result, arg)
    return result

#### Logical Operations

In [ ]:
# rotating bitstring left x = x1||x2, x <<< len(x1) = x2 || x1 
def rotateLeft(input, places):
    p = places % len(input)
    return input[-p:] + input[:-p]

# rotating bitstring right x = x1||x2, x >>> len(x2) = x2 || x1 
def rotateRight(input, places):
    return rotateLeft(input, -places)

# shifting bitstring left x << 2 = 00 || x[:2]
def shiftLeft(input, p):
    if abs(p) >= len(input):
        # Everything gets shifted out anyway
        return "0" * len(input)
    if p < 0:
        # Shift right instead
        return  input[-p:] + "0" * len(input[:-p])
    elif p == 0:
        return input
    else: # p > 0, normal case
        return "0" * len(input[-p:]) + input[:-p]

# shifting bitstring right x >> 2 = x[2:] || 0
def shiftRight(input, p):
    return shiftLeft(input, -p)

# returns length of key string
def keyLengthInBitsOf(k):
    return len(k) * 4

### Data Tables

#### S-boxes

In [ ]:
# all 8 S-boxes in one table
SBoxDecimalTable = [
	[3, 8, 15, 1, 10, 6, 5, 11, 14, 13, 4, 2, 7, 0, 9, 12], # S0
	[15, 12, 2, 7, 9, 0, 5, 10, 1, 11, 14, 8, 6, 13, 3, 4], # S1
	[8, 6, 7, 9, 3, 12, 10, 15, 13, 1, 14, 4, 0, 11, 5, 2], # S2
	[0, 15, 11, 8, 12, 9, 6, 3, 13, 1, 2, 4, 10, 7, 5, 14], # S3
	[1, 15, 8, 3, 12, 0, 11, 6, 2, 5, 4, 10, 9, 14, 7, 13], # S4
	[15, 5, 2, 11, 4, 10, 9, 12, 0, 3, 14, 8, 13, 6, 7, 1], # S5
	[7, 2, 12, 5, 8, 4, 6, 11, 14, 9, 1, 15, 13, 3, 10, 0], # S6
	[1, 13, 15, 0, 14, 8, 2, 11, 7, 4, 12, 10, 9, 3, 5, 6], # S7
    ] 

# S-boxes as dictionaries and inverse S-boxes 
# SboxBitString has x and S-box[x] as bit strings. Therefore list of 8 dictionaries
SBoxBitstring = []
# SBoxBitstringInverse has x and Sinv[x] as bit strings. Therefore, list of 8 dictionaries
SBoxBitstringInverse = []
for line in SBoxDecimalTable:
    dict = {}
    inverseDict = {}
    for i in range(len(line)):
        index = bitstring(i, 4)
        value = bitstring(line[i], 4)
        dict[index] = value
        inverseDict[value] = index
    SBoxBitstring.append(dict)
    SBoxBitstringInverse.append(inverseDict)

In [ ]:
print(SBoxBitstring)

[{'0000': '0011', '0001': '1000', '0010': '1111', '0011': '0001', '0100': '1010', '0101': '0110', '0110': '0101', '0111': '1011', '1000': '1110', '1001': '1101', '1010': '0100', '1011': '0010', '1100': '0111', '1101': '0000', '1110': '1001', '1111': '1100'}, {'0000': '1111', '0001': '1100', '0010': '0010', '0011': '0111', '0100': '1001', '0101': '0000', '0110': '0101', '0111': '1010', '1000': '0001', '1001': '1011', '1010': '1110', '1011': '1000', '1100': '0110', '1101': '1101', '1110': '0011', '1111': '0100'}, {'0000': '1000', '0001': '0110', '0010': '0111', '0011': '1001', '0100': '0011', '0101': '1100', '0110': '1010', '0111': '1111', '1000': '1101', '1001': '0001', '1010': '1110', '1011': '0100', '1100': '0000', '1101': '1011', '1110': '0101', '1111': '0010'}, {'0000': '0000', '0001': '1111', '0010': '1011', '0011': '1000', '0100': '1100', '0101': '1001', '0110': '0110', '0111': '0011', '1000': '1101', '1001': '0001', '1010': '0010', '1011': '0100', '1100': '1010', '1101': '0111', 

#### Permutations

In [ ]:
# shows which value comes to which position. 
# Having value v(say, 32) at position p (say, 1) means that the output bit at position p(1) comes from the input bit at position v (32).

# Table for initial permutation
IPTable = [
    0, 32, 64, 96, 1, 33, 65, 97, 2, 34, 66, 98, 3, 35, 67, 99,
    4, 36, 68, 100, 5, 37, 69, 101, 6, 38, 70, 102, 7, 39, 71, 103,
    8, 40, 72, 104, 9, 41, 73, 105, 10, 42, 74, 106, 11, 43, 75, 107,
    12, 44, 76, 108, 13, 45, 77, 109, 14, 46, 78, 110, 15, 47, 79, 111,
    16, 48, 80, 112, 17, 49, 81, 113, 18, 50, 82, 114, 19, 51, 83, 115,
    20, 52, 84, 116, 21, 53, 85, 117, 22, 54, 86, 118, 23, 55, 87, 119,
    24, 56, 88, 120, 25, 57, 89, 121, 26, 58, 90, 122, 27, 59, 91, 123,
    28, 60, 92, 124, 29, 61, 93, 125, 30, 62, 94, 126, 31, 63, 95, 127,
    ]

# Table for final permutation
FPTable = [
    0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60,
    64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116, 120, 124,
    1, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61,
    65, 69, 73, 77, 81, 85, 89, 93, 97, 101, 105, 109, 113, 117, 121, 125,
    2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62,
    66, 70, 74, 78, 82, 86, 90, 94, 98, 102, 106, 110, 114, 118, 122, 126,
    3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63,
    67, 71, 75, 79, 83, 87, 91, 95, 99, 103, 107, 111, 115, 119, 123, 127,
 ]

#### Linear Transformation

In [ ]:
# the linear transformation table consists of 128 lists that consist the position of elements in input block that need to be xored in order to get corresponding element of output block.
# NON-BITSLICE MODE

# table for linear transformation used while encryption
LTTable = [
    [16, 52, 56, 70, 83, 94, 105],
    [72, 114, 125],
    [2, 9, 15, 30, 76, 84, 126],
    [36, 90, 103],
    [20, 56, 60, 74, 87, 98, 109],
    [1, 76, 118],
    [2, 6, 13, 19, 34, 80, 88],
    [40, 94, 107],
    [24, 60, 64, 78, 91, 102, 113],
    [5, 80, 122],
    [6, 10, 17, 23, 38, 84, 92],
    [44, 98, 111],
    [28, 64, 68, 82, 95, 106, 117],
    [9, 84, 126],
    [10, 14, 21, 27, 42, 88, 96],
    [48, 102, 115],
    [32, 68, 72, 86, 99, 110, 121],
    [2, 13, 88],
    [14, 18, 25, 31, 46, 92, 100],
    [52, 106, 119],
    [36, 72, 76, 90, 103, 114, 125],
    [6, 17, 92],
    [18, 22, 29, 35, 50, 96, 104],
    [56, 110, 123],
    [1, 40, 76, 80, 94, 107, 118],
    [10, 21, 96],
    [22, 26, 33, 39, 54, 100, 108],
    [60, 114, 127],
    [5, 44, 80, 84, 98, 111, 122],
    [14, 25, 100],
    [26, 30, 37, 43, 58, 104, 112],
    [3, 118],
    [9, 48, 84, 88, 102, 115, 126],
    [18, 29, 104],
    [30, 34, 41, 47, 62, 108, 116],
    [7, 122],
    [2, 13, 52, 88, 92, 106, 119],
    [22, 33, 108],
    [34, 38, 45, 51, 66, 112, 120],
    [11, 126],
    [6, 17, 56, 92, 96, 110, 123],
    [26, 37, 112],
    [38, 42, 49, 55, 70, 116, 124],
    [2, 15, 76],
    [10, 21, 60, 96, 100, 114, 127],
    [30, 41, 116],
    [0, 42, 46, 53, 59, 74, 120],
    [6, 19, 80],
    [3, 14, 25, 100, 104, 118],
    [34, 45, 120],
    [4, 46, 50, 57, 63, 78, 124],
    [10, 23, 84],
    [7, 18, 29, 104, 108, 122],
    [38, 49, 124],
    [0, 8, 50, 54, 61, 67, 82],
    [14, 27, 88],
    [11, 22, 33, 108, 112, 126],
    [0, 42, 53],
    [4, 12, 54, 58, 65, 71, 86],
    [18, 31, 92],
    [2, 15, 26, 37, 76, 112, 116],
    [4, 46, 57],
    [8, 16, 58, 62, 69, 75, 90],
    [22, 35, 96],
    [6, 19, 30, 41, 80, 116, 120],
    [8, 50, 61],
    [12, 20, 62, 66, 73, 79, 94],
    [26, 39, 100],
    [10, 23, 34, 45, 84, 120, 124],
    [12, 54, 65],
    [16, 24, 66, 70, 77, 83, 98],
    [30, 43, 104],
    [0, 14, 27, 38, 49, 88, 124],
    [16, 58, 69],
    [20, 28, 70, 74, 81, 87, 102],
    [34, 47, 108],
    [0, 4, 18, 31, 42, 53, 92],
    [20, 62, 73],
    [24, 32, 74, 78, 85, 91, 106],
    [38, 51, 112],
    [4, 8, 22, 35, 46, 57, 96],
    [24, 66, 77],
    [28, 36, 78, 82, 89, 95, 110],
    [42, 55, 116],
    [8, 12, 26, 39, 50, 61, 100],
    [28, 70, 81],
    [32, 40, 82, 86, 93, 99, 114],
    [46, 59, 120],
    [12, 16, 30, 43, 54, 65, 104],
    [32, 74, 85],
    [36, 90, 103, 118],
    [50, 63, 124],
    [16, 20, 34, 47, 58, 69, 108],
    [36, 78, 89],
    [40, 94, 107, 122],
    [0, 54, 67],
    [20, 24, 38, 51, 62, 73, 112],
    [40, 82, 93],
    [44, 98, 111, 126],
    [4, 58, 71],
    [24, 28, 42, 55, 66, 77, 116],
    [44, 86, 97],
    [2, 48, 102, 115],
    [8, 62, 75],
    [28, 32, 46, 59, 70, 81, 120],
    [48, 90, 101],
    [6, 52, 106, 119],
    [12, 66, 79],
    [32, 36, 50, 63, 74, 85, 124],
    [52, 94, 105],
    [10, 56, 110, 123],
    [16, 70, 83],
    [0, 36, 40, 54, 67, 78, 89],
    [56, 98, 109],
    [14, 60, 114, 127],
    [20, 74, 87],
    [4, 40, 44, 58, 71, 82, 93],
    [60, 102, 113],
    [3, 18, 72, 114, 118, 125],
    [24, 78, 91],
    [8, 44, 48, 62, 75, 86, 97],
    [64, 106, 117],
    [1, 7, 22, 76, 118, 122],
    [28, 82, 95],
    [12, 48, 52, 66, 79, 90, 101],
    [68, 110, 121],
    [5, 11, 26, 80, 122, 126],
    [32, 86, 99],
    ]

# Table for linear transformation while decryption
LTTableInverse = [
    [53, 55, 72],
    [1, 5, 20, 90],
    [15, 102],
    [3, 31, 90],
    [57, 59, 76],
    [5, 9, 24, 94],
    [19, 106],
    [7, 35, 94],
    [61, 63, 80],
    [9, 13, 28, 98],
    [23, 110],
    [11, 39, 98],
    [65, 67, 84],
    [13, 17, 32, 102],
    [27, 114],
    [1, 3, 15, 20, 43, 102],
    [69, 71, 88],
    [17, 21, 36, 106],
    [1, 31, 118],
    [5, 7, 19, 24, 47, 106],
    [73, 75, 92],
    [21, 25, 40, 110],
    [5, 35, 122],
    [9, 11, 23, 28, 51, 110],
    [77, 79, 96],
    [25, 29, 44, 114],
    [9, 39, 126],
    [13, 15, 27, 32, 55, 114],
    [81, 83, 100],
    [1, 29, 33, 48, 118],
    [2, 13, 43],
    [1, 17, 19, 31, 36, 59, 118],
    [85, 87, 104],
    [5, 33, 37, 52, 122],
    [6, 17, 47],
    [5, 21, 23, 35, 40, 63, 122],
    [89, 91, 108],
    [9, 37, 41, 56, 126],
    [10, 21, 51],
    [9, 25, 27, 39, 44, 67, 126],
    [93, 95, 112],
    [2, 13, 41, 45, 60],
    [14, 25, 55],
    [2, 13, 29, 31, 43, 48, 71],
    [97, 99, 116],
    [6, 17, 45, 49, 64],
    [18, 29, 59],
    [6, 17, 33, 35, 47, 52, 75],
    [101, 103, 120],
    [10, 21, 49, 53, 68],
    [22, 33, 63],
    [10, 21, 37, 39, 51, 56, 79],
    [105, 107, 124],
    [14, 25, 53, 57, 72],
    [26, 37, 67],
    [14, 25, 41, 43, 55, 60, 83],
    [0, 109, 111],
    [18, 29, 57, 61, 76],
    [30, 41, 71],
    [18, 29, 45, 47, 59, 64, 87],
    [4, 113, 115],
    [22, 33, 61, 65, 80],
    [34, 45, 75],
    [22, 33, 49, 51, 63, 68, 91],
    [8, 117, 119],
    [26, 37, 65, 69, 84],
    [38, 49, 79],
    [26, 37, 53, 55, 67, 72, 95],
    [12, 121, 123],
    [30, 41, 69, 73, 88],
    [42, 53, 83],
    [30, 41, 57, 59, 71, 76, 99],
    [16, 125, 127],
    [34, 45, 73, 77, 92],
    [46, 57, 87],
    [34, 45, 61, 63, 75, 80, 103],
    [1, 3, 20],
    [38, 49, 77, 81, 96],
    [50, 61, 91],
    [38, 49, 65, 67, 79, 84, 107],
    [5, 7, 24],
    [42, 53, 81, 85, 100],
    [54, 65, 95],
    [42, 53, 69, 71, 83, 88, 111],
    [9, 11, 28],
    [46, 57, 85, 89, 104],
    [58, 69, 99],
    [46, 57, 73, 75, 87, 92, 115],
    [13, 15, 32],
    [50, 61, 89, 93, 108],
    [62, 73, 103],
    [50, 61, 77, 79, 91, 96, 119],
    [17, 19, 36],
    [54, 65, 93, 97, 112],
    [66, 77, 107],
    [54, 65, 81, 83, 95, 100, 123],
    [21, 23, 40],
    [58, 69, 97, 101, 116],
    [70, 81, 111],
    [58, 69, 85, 87, 99, 104, 127],
    [25, 27, 44],
    [62, 73, 101, 105, 120],
    [74, 85, 115],
    [3, 62, 73, 89, 91, 103, 108],
    [29, 31, 48],
    [66, 77, 105, 109, 124],
    [78, 89, 119],
    [7, 66, 77, 93, 95, 107, 112],
    [33, 35, 52],
    [0, 70, 81, 109, 113],
    [82, 93, 123],
    [11, 70, 81, 97, 99, 111, 116],
    [37, 39, 56],
    [4, 74, 85, 113, 117],
    [86, 97, 127],
    [15, 74, 85, 101, 103, 115, 120],
    [41, 43, 60],
    [8, 78, 89, 117, 121],
    [3, 90],
    [19, 78, 89, 105, 107, 119, 124],
    [45, 47, 64],
    [12, 82, 93, 121, 125],
    [7, 94],
    [0, 23, 82, 93, 109, 111, 123],
    [49, 51, 68],
    [1, 16, 86, 97, 125],
    [11, 98],
    [4, 27, 86, 97, 113, 115, 127],
]

#### Constants

In [ ]:
# Fractional part of Golden Ratio
phi = 0x9e3779b9

# Number of rounds
r = 32

### Functions

#### Substitution step

In [ ]:
# S-box and S-box inverse
# substitution according to provided box number
def S(box, input):
    return SBoxBitstring[box%8][input]
    
# inverse substitution according to provided block number
def SInverse(box, output):
    return SBoxBitstringInverse[box%8][output]


# Substituting each character in 128 bit string according to box number provided 
# substitution
def SHat(box, input):
    result = ""
    for i in range(32):
        result = result + S(box, input[4*i:4*(i+1)])
    return result

# inverse substitution
def SHatInverse(box, output):
    result = ""
    for i in range(32):
        result = result + SInverse(box, output[4*i:4*(i+1)])
    return result


# Substituting each character of 4 32 bit words taken as list
# substitution
def SBitslice(box, words):
    result = ["", "", "", ""]
    for i in range(32):
        quad = S(box, words[0][i] + words[1][i] + words[2][i] + words[3][i])
        for j in range(4):
            result[j] = result[j] + quad[j]
    return result

# inverse substitution
def SBitsliceInverse(box, words):
    result = ["", "", "", ""]
    for i in range(32): 
        quad = SInverse(
            box, words[0][i] + words[1][i] + words[2][i] + words[3][i])
        for j in range(4):
            result[j] = result[j] + quad[j]
    return result

#### Permutation step

In [ ]:
# USing specified permutation table (IP or LP) as specified to perform permutation on 128 bit string
def applyPermutation(permutationTable, input):
    if len(input) != len(permutationTable):
        print("ValueError: input size {} doesn't match perm table size {}".format(len(input), len(permutationTable)))
    result = ""
    for i in range(len(permutationTable)):
        result = result + input[permutationTable[i]]
    return result

# Initial permutation
def IP(input):
    """Apply the Initial Permutation to the 128-bit bitstring 'input'
    and return a 128-bit bitstring as the result."""
    return applyPermutation(IPTable, input)

# Inverse initial permutation or final permutation
def IPInverse(output):
    return FP(output)

# Final permutation
def FP(input):
    return applyPermutation(FPTable, input)

# Inverse final permutation or initial permutation
def FPInverse(output):
    """Apply the Final Permutation in reverse."""
    return IP(output)

#### Linear transformation step

In [ ]:
# Using table to perform linear transformation on 128 bit string and returning 128 bit output
# linear transformation
def LT(input):
    if len(input) != 128:
        print("ValueError: input to LT is not 128 bit long")
    result = ""
    for i in range(len(LTTable)):
        outputBit = "0"
        for j in LTTable[i]:
            outputBit = xor(outputBit, input[j])
        result = result + outputBit
    return result
    
# inverse linear transformation
def LTInverse(output):
    if len(output) != 128:
        print("ValueError: input to inverse LT is not 128 bit long")
    result = ""
    for i in range(len(LTTableInverse)):
        inputBit = "0"
        for j in LTTableInverse[i]:
            inputBit = xor(inputBit, output[j])
        result = result + inputBit
    return result


# Using the linear transformation equation to the 4 32 bit input words, least significant word's bitstring taken first
# linear transformation
def LTBitslice(X):
    X[0] = rotateLeft(X[0], 13)
    X[2] = rotateLeft(X[2], 3)
    X[1] = xor(X[1], X[0], X[2])
    X[3] = xor(X[3], X[2], shiftLeft(X[0], 3))
    X[1] = rotateLeft(X[1], 1)
    X[3] = rotateLeft(X[3], 7)
    X[0] = xor(X[0], X[1], X[3])
    X[2] = xor(X[2], X[3], shiftLeft(X[1], 7))
    X[0] = rotateLeft(X[0], 5)
    X[2] = rotateLeft(X[2], 22)
    return X

# inverse linear transformation
def LTBitsliceInverse(X):
    X[2] = rotateRight(X[2], 22)
    X[0] = rotateRight(X[0], 5)
    X[2] = xor(X[2], X[3], shiftLeft(X[1], 7))
    X[0] = xor(X[0], X[1], X[3])
    X[3] = rotateRight(X[3], 7)
    X[1] = rotateRight(X[1], 1)
    X[3] = xor(X[3], X[2], shiftLeft(X[0], 3))
    X[1] = xor(X[1], X[0], X[2])
    X[2] = rotateRight(X[2], 3)
    X[0] = rotateRight(X[0], 13)
    return X

### Round

In [ ]:
# Each round takes a 128 bit string and applies the round operations on it in the given order: 
# round key mixing, substitution and linear transformation (addition key mixing in case of last round)
# r = 32 rounds
# one round during encryption
def R(i, BHati, KHat):
    # adding round key
    xored = xor(BHati, KHat[i])
    # Passing through sbox
    SHati = SHat(i, xored)
    # Last step of round depending on which rounkd it is
    # for every round except last, linear transformation applied
    if 0 <= i <= r-2:
        BHatiPlus1 = LT(SHati)
    # in case of last round, round key added
    elif i == r-1:
        BHatiPlus1 = xor(SHati, KHat[r])
    else:
        print("ValueError: round {} is out of 0..{} range".format(i, r-1))
    # returning input for the next round except in case of last round
    return BHatiPlus1

# Each round takes a 128 bit string and applies the round operation on it in the given order:
# inverse linear transformation (additional key mixing in case of first decryption round = inverse of last encryption round), inverse substitution, key mixing
# r = 32 rounds
# one round during decryption
def RInverse(i, BHatiPlus1, KHat): 
    # inverse linear transformation
    if 0 <= i <= r-2:
        SHati = LTInverse(BHatiPlus1)
    # in case of inverse of last encryption round
    elif i == r-1:
        SHati = xor(BHatiPlus1, KHat[r])
    else:
        print("ValueError: round {} is out of 0..{} range".format(i, r-1))
    # inverse substitution
    xored = SHatInverse(i, SHati)
    # key mixing
    BHati = xor(xored, KHat[i])
    # returns input for previous round
    return BHati


# Bitslice version (input and output are both lists of 4 32-bit bitstrings)
def RBitslice(i, Bi, K):
    # Key mixing
    xored = xor (Bi, K[i])
    # Substitution
    Si = SBitslice(i, quadSplit(xored))
    # Linear Transformation
    if i == r-1:
        # In the last round, replaced by an additional key mixing
        BiPlus1 = xor(quadJoin(Si), K[r])
    else:
        BiPlus1 = quadJoin(LTBitslice(Si))
    # BIPlus1 is a 128-bit bitstring
    return BiPlus1

# Bitslice version (input and output are both lists of 4 32-bit bitstrings)
def RBitsliceInverse(i, BiPlus1, K):
    # Linear Transformation
    if i == r-1:
        # In the last round, replaced by an additional key mixing
        Si = quadSplit(xor(BiPlus1, K[r]))
    else:
        Si = LTBitsliceInverse(quadSplit(BiPlus1))
    # S Boxes
    xored = SBitsliceInverse(i, Si)
    # Key mixing
    Bi = xor (quadJoin(xored), K[i])
    return Bi

### Keyscheduling

In [ ]:
def makeSubkeys(userKey):
    # We write the key as 8 32-bit words w-8 ... w-1
    # w-8 is the least significant word
    w = {}
    for i in range(-8, 0):
        w[i] = userKey[(i+8)*32:(i+9)*32]


    # We expand these to a prekey w0 ... w131 with the affine recurrence
    for i in range(132):
        w[i] = rotateLeft(
            xor(w[i-8], w[i-5], w[i-3], w[i-1], bitstring(phi, 32), bitstring(i,32)), 11)


    # The round keys are now calculated from the prekeys using the S-boxes
    # in bitslice mode. Each k[i] is a 32-bit bitstring.
    k = {}
    for i in range(r+1):
        whichS = (r + 3 - i) % r
        k[0+4*i] = ""
        k[1+4*i] = ""
        k[2+4*i] = ""
        k[3+4*i] = ""
        for j in range(32): # for every bit in the k and w words
            # ENOTE: w0 and k0 are the least significant words, w99 and k99
            # the most.
            input = w[0+4*i][j] + w[1+4*i][j] + w[2+4*i][j] + w[3+4*i][j]
            output = S(whichS, input)
            for l in range(4):
                k[l+4*i] = k[l+4*i] + output[l]

    # We then renumber the 32 bit values k_j as 128 bit subkeys K_i.
    K = []
    for i in range(33):
        # ENOTE: k4i is the least significant word, k4i+3 the most.
        K.append(k[4*i] + k[4*i+1] + k[4*i+2] + k[4*i+3])

    # We now apply IP to the round key in order to place the key bits in
    # the correct column
    KHat = []
    for i in range(33):
        KHat.append(IP(K[i]))
        #O.show("Ki", K[i], "(i=%2d) Ki" % i)
        #O.show("KHati", KHat[i], "(i=%2d) KHati" % i)

    return w, K, KHat


def makeLongKey(k):
    """Take a key k in bitstring format. Return the long version of that
    key."""

    l = len(k)
    if l % 32 != 0 or l < 64 or l > 256:
        print("ValueError: Invalid key length {} bits)".format(l))
    
    if l == 256:
        return k
    else:
        return k + "1" + "0"*(256 -l -1)

In [ ]:
key = "6E3272357538782F413F4428472B4B6250645367566B59703373367639792442"
bitkey = bitstring(int(key, 16), 256)
print(bitkey)
subkeys = makeSubkeys(bitkey)
hexkey = hex(int(bitkey, 2))[2:].upper()
print(hexkey == key)

0110111000110010011100100011010101110101001110000111100000101111010000010011111101000100001010000100011100101011010010110110001001010000011001000101001101100111010101100110101101011001011100000011001101110011001101100111011000111001011110010010010001000010
True


In [ ]:
key = "6E3272397538789F413F4428472B4B6250645367566B59703373367639792442"
bitkey = bitstring(int(key, 16), 256)
print(bitkey)
subkeys = makeSubkeys(bitkey)
hexkey = hex(int(bitkey, 2))[2:].upper()
print(hexkey == key)

0110111000110010011100100011100101110101001110000111100010011111010000010011111101000100001010000100011100101011010010110110001001010000011001000101001101100111010101100110101101011001011100000011001101110011001101100111011000111001011110010010010001000010
True


### Encryption

In [ ]:
# input : 128-bit string of plaintext and 256-bit string of key in bits
# outputs 128 bit string of ciphertext in bits
def encrypt(plainText, userKey):   
    # Key scheduling to get sub keys for each round
    w, K, KHat = makeSubkeys(userKey)
    # Initial permutation
    BHat = IP(plainText) 
    # 32 rounds
    for i in range(r):
        BHat = R(i, BHat, KHat) # Produce BHat_i+1 from BHat_i
    # BHat is now _32 i.e. _r
    # Final permutation
    C = FP(BHat)
    # Ciphertext
    return C

# Bitslice version 
def encryptBitslice(plainText, userKey):
    K, KHat = makeSubkeys(userKey)
    B = plainText 
    for i in range(r):
        B = RBitslice(i, B, K) 
    return B

### Decryption

In [ ]:
# input : 128-bit string of ciphertext and 256-bit string of key in bits
# outputs 128-bit string of plaintext in bits
def decrypt(cipherText, userKey):
    # key scheduling to get subkeys for all rounds
    w, K, KHat = makeSubkeys(userKey)
    # Inverse of final permutation => initial permutation
    BHat = FPInverse(cipherText) # BHat_r at this stage
    # Inverse rounds
    for i in range(r-1, -1, -1): # from r-1 down to 0 included
        BHat = RInverse(i, BHat, KHat)
    # Inverse of initial permutation => final permutation
    plainText = IPInverse(BHat)
    # returns plaintext
    return plainText
    
# Bitslice version
def decryptBitslice(cipherText, userKey):
    K, KHat = makeSubkeys(userKey)
    B = cipherText 
    for i in range(r-1, -1, -1):
        B = RBitsliceInverse(i, B, K) 
    return B

### To-Use

In [ ]:
process = int(input("Would you like to perform encryption or decryption (1:encryption / 2:decryption)? "))
all = []

def Process(process):
  text = input("Please enter 128 bit text in hexadecimal format: ")
  bittext = bin(int("0x"+text, 16))
  key = input("Please enter 256 bit key in hexadecimal format: ")
  bitkey = bin(int("0x"+key, 16))
  if process == 1:
      encrypted = encrypt(bittext, bitkey)
      hexenc = str(hex(int(encrypted, 2)))[2:]
      print("\n\ngiven plaintext: {}\nciphertext: {}\n\n".format(text, hexenc.upper()))
      all.append(encrypted)
      all.append(hexenc)
  elif process == 2:
      decrypted = decrypt(bittext, bitkey) 
      hexdec = str(hex(int(decrypted, 2)))[2:] 
      print("\n\ngiven ciphertext: {}\nplaintext: {}\n\n".format(text, hexdec.upper()))
      all.append(decrypted)
      all.append(hexdec)
  else:
      print("\n\nPlease enter a valid choice.\n\n")
  cont = input("Do you want to continue (Y:yes / N:no)? ")
  if cont == "Y":
      pn = int(input("\n\nWould you like to perform encryption or decryption (1:encryption / 2:decryption)? "))
      Process(pn)
  return all

info = Process(process)

#Integral Property




In [ ]:
def R_small_add(i, BHati, KHat):
    # adding round key
    xored = xor(BHati, KHat[i])
    # Passing through sbox
    SHati = SHat(i, xored)
    # Last step of round depending on which rounkd it is
    # for every round except last, linear transformation applied
    if 0 <= i <= r-2:
        BHatiPlus1 = LT(SHati)
    # in case of last round, round key added
    elif i == r-1:
        BHatiPlus1 = xor(SHati, KHat[r])
    else:
        print("ValueError: round {} is out of 0..{} range".format(i, r-1))
    # returning input for the next round except in case of last round
    return SHati, BHatiPlus1

In [ ]:
def round_encrypt(plainText, userKey):   
    half_round_result = []
    round_result = []
    # Key scheduling to get sub keys for each round
    w, K, KHat = makeSubkeys(userKey)
    # Initial permutation
    BHat = plainText 
    # 32 rounds
    for i in range(r):
        after_sub = R_small_add(i, BHat, KHat)[0]
        if len(after_sub) < 128:
          after_sub = "0"*(128-len(after_sub)) + after_sub
        BHat = R_small_add(i, BHat, KHat)[1] # Produce BHat_i+1 from BHat_i
        if len(BHat) < 128:
          BHat = "0"*(128-len(BHat)) + BHat
        round_result.append(BHat)
        half_round_result.append(after_sub)
    # BHat is now _32 i.e. _r
    # Final permutation
    C = BHat
    # Ciphertext
    return round_result, C, half_round_result

In [ ]:
key = "F0655368566D5971337436763979244226452948404D635166546A576E5A7234"
bitkey = bitstring(int(key, 16), 256)
print(bitkey)

1111000001100101010100110110100001010110011011010101100101110001001100110111010000110110011101100011100101111001001001000100001000100110010001010010100101001000010000000100110101100011010100010110011001010100011010100101011101101110010110100111001000110100


In [ ]:
key = "357538782F413F4428472B4B6250655368566D59713374367639792442264529"
bitkey = bitstring(int(key, 16), 256)
print(bitkey)

0011010101110101001110000111100000101111010000010011111101000100001010000100011100101011010010110110001001010000011001010101001101101000010101100110110101011001011100010011001101110100001101100111011000111001011110010010010001000010001001100100010100101001


In [ ]:
initial_message = "28472B4B625065536856FD5971337436"
messages = []
for i in range(16):
    new_msg = hex(i)[2:] + initial_message[1:]
    messages.append(new_msg.upper())
messages

['08472B4B625065536856FD5971337436',
 '18472B4B625065536856FD5971337436',
 '28472B4B625065536856FD5971337436',
 '38472B4B625065536856FD5971337436',
 '48472B4B625065536856FD5971337436',
 '58472B4B625065536856FD5971337436',
 '68472B4B625065536856FD5971337436',
 '78472B4B625065536856FD5971337436',
 '88472B4B625065536856FD5971337436',
 '98472B4B625065536856FD5971337436',
 'A8472B4B625065536856FD5971337436',
 'B8472B4B625065536856FD5971337436',
 'C8472B4B625065536856FD5971337436',
 'D8472B4B625065536856FD5971337436',
 'E8472B4B625065536856FD5971337436',
 'F8472B4B625065536856FD5971337436']

In [ ]:
initial_message = "28472B4B665525036856F09876547436"
messages = []
for i in range(16):
    new_msg = hex(i)[2:] + initial_message[1:]
    messages.append(new_msg.upper())
messages

['08472B4B665525036856F09876547436',
 '18472B4B665525036856F09876547436',
 '28472B4B665525036856F09876547436',
 '38472B4B665525036856F09876547436',
 '48472B4B665525036856F09876547436',
 '58472B4B665525036856F09876547436',
 '68472B4B665525036856F09876547436',
 '78472B4B665525036856F09876547436',
 '88472B4B665525036856F09876547436',
 '98472B4B665525036856F09876547436',
 'A8472B4B665525036856F09876547436',
 'B8472B4B665525036856F09876547436',
 'C8472B4B665525036856F09876547436',
 'D8472B4B665525036856F09876547436',
 'E8472B4B665525036856F09876547436',
 'F8472B4B665525036856F09876547436']

In [ ]:
initial_message = "28472B4B665525036856F09876547436"
messages = []
for i in range(16):
    new_msg = hex(i)[2:] + initial_message[1:]
    messages.append(new_msg.upper())
messages

['08472B4B665525036856F09876547436',
 '18472B4B665525036856F09876547436',
 '28472B4B665525036856F09876547436',
 '38472B4B665525036856F09876547436',
 '48472B4B665525036856F09876547436',
 '58472B4B665525036856F09876547436',
 '68472B4B665525036856F09876547436',
 '78472B4B665525036856F09876547436',
 '88472B4B665525036856F09876547436',
 '98472B4B665525036856F09876547436',
 'A8472B4B665525036856F09876547436',
 'B8472B4B665525036856F09876547436',
 'C8472B4B665525036856F09876547436',
 'D8472B4B665525036856F09876547436',
 'E8472B4B665525036856F09876547436',
 'F8472B4B665525036856F09876547436']

In [ ]:
round1 = []
round2 = []
round2_5 = []
round3 = []
round4 = []
round_results_all = []

for i in range(len(messages)):
    round_bit = bin(int(messages[i], 16))[2:]
    if len(round_bit) < 128:
      round_bit = "0"*(128-len(round_bit)) + round_bit
    results = round_encrypt(round_bit, bitkey)
    result = results[0]
    half_result = results[2]
    round1.append(hex(int(result[0], 2))[2:])
    round2.append(hex(int(result[1], 2))[2:])
    round3.append(hex(int(result[2], 2))[2:])
    round4.append(hex(int(result[3], 2))[2:])
    round2_5.append(hex(int(half_result[2], 2))[2:])
    #round1.append(result[0])   
    round_results_all.append(result) 

In [ ]:
round1

['1e3966d08ea0c50f9e9ca0f10225af08',
 '3c3926d186b247479e14a0f000252d08',
 '1e3966d18ea2474f9e14a0f002252d08',
 '3839265186b045079e9ca0f10025ad28',
 '3839265086b2c7479e14a0f000252f28',
 '3c3926d086b0c5079e9ca0f10025af08',
 '3c3926d186b045079e9ca0f10025ad08',
 '1a3966518ea2474f9e14a0f002252d28',
 '1a3966518ea0450f9e9ca0f10225ad28',
 '3839265086b0c5079e9ca0f10025af28',
 '3c3926d086b2c7479e14a0f000252f08',
 '1a3966508ea2c74f9e14a0f002252f28',
 '1e3966d08ea2c74f9e14a0f002252f08',
 '3839265186b247479e14a0f000252d28',
 '1a3966508ea0c50f9e9ca0f10225af28',
 '1e3966d18ea0450f9e9ca0f10225ad08']

In [ ]:
#round1
prop = []
for i in range(32):
  listmade = []
  for j in range(16):
    listmade.append(round1[j][i])
  setlen = len(set(listmade))
  prop.append(setlen)
print(prop)

[2, 4, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1]


In [ ]:
round2

['4d91715ba60401a8c114a4a13e84d5e2',
 '6fa1bb55fe0ec3449fc56745917cca1a',
 '4480f9e0047ce732eee8e5c6c05ceaba',
 '4279a261d6f601e2b3baf6827dc6feac',
 '4b682ada748ee7789c46b7e5831ec1f4',
 '66b033ee5c7625deb03926226fa4f542',
 '42b9036dd676a1c6b1b03682e5c4f7aa',
 '444058ec14fc4f16ece225c6585ee39c',
 '6958e0d43c842d94c29774012ce6de2c',
 '667092e25cf685fab233e622f7a6fc44',
 '4ba88bd6740e475c9e4c77e51b1cc8f2',
 '6049686f9efccb0eed6b3566d23ee174',
 '6089c9638e7c632aef61f5664a3ce852',
 '6f611a59fe8e63609dcfa745097ec31c',
 '4d51d057b684a98cc31e64a1a686dcc4',
 '699841d82c0485b0c09db401b4e4d70a']

In [ ]:
round2_5

['a6e8cdcdcded2bdacda6963f24af3d65',
 '391817ca88e792ae8a4827e9c312cf5a',
 'ad46b8592eb86460ff1bf8788c42ff3a',
 'a53dd4a77dce2b007e02a9a34d7ca9c8',
 'afc189b1de7764ca8c9977893a2bc08d',
 '3ed66e5be7beb12871304903ee3fad45',
 'a5dd7ea57dbecb5d7d0ce9a3787faeca',
 'adf6f6525ec8499dfc1f48781b4bf5a8',
 '3ca101bf077db64ec457565f518c39b8',
 '3e364456e7cef1b3743af903973ca64d',
 'af11e7bcdee744f78f915789b522cc85',
 '30fd36a0a8c89f78f365e8d86fbbf0dd',
 '304d58a8f8b81283fa63a8d8d2b2fce5',
 '39c899ce88771235834e97e9001bc558',
 'a6a821c33d7dcce7cea4263fa9ac369d',
 '3ce1adb4b7edf115c15d765ff68f3e2a']

In [ ]:
round4

['b6096593bec75cec81f5d0f317c82203',
 'aef1291f8ea28b01c467fcef5b89841c',
 'da21ac64d1df3ed1fc792014173bf797',
 'f1e4bc7dd50928b495f5843b20e22bc7',
 'f9ea384657963ffced71b137dcabf066',
 '13874705970fcb5ba32d523afc9069ed',
 '971d53d4f94203f7e4beb12465492b19',
 'bcea07bc4acca6e1b8fc4faac5a5206b',
 'f8b7b08d7a79a2f3316d604538be318a',
 'c0b5ef8615a9963dd7a1f52b1801b6af',
 'ac3650f0be7ed04b080e3e75d489958b',
 'c29d23ecf423a18033f995c7be81230d',
 'efbf9623f387ef88f0817c1383284bf4',
 'f2f3976dac71f6ffe1a8a981a56b2465',
 '474d033ee6558986bf60060c1d5b64c1',
 '3846734641183a6cfe61d8fc3371d7fe']

In [ ]:
xored1 =[]
xored2 = []
xored3 = []
xored4 = 0
xored2_5 = []

for j in range(32):
  xored = 0
  for i in range(len(round1)):
    xored = xored ^ int(round1[i][j], 16)
  xored1.append(xored)

for k in range(32):
  xored = 0
  for l in range(len(round2)):
    xored = xored ^ int(round2[l][k], 16)
  xored2.append(xored)

#for m in range(len(round3[0])):
#  xored = 0
#  for n in range(len(round3)):
#    xored = xored ^ int(round3[n][m], 16)
#  xored3.append(xored)

#for q in range(len(round4)):
#  xored4 = xored4 ^ int(round4[q][0], 16)

for o in range(len(round2_5[0])):
  xored = 0
  for p in range(len(round2_5)):
    xored = xored ^ int(round2_5[p][o], 16)
  xored2_5.append(xored)

In [ ]:
print(xored1)
print(xored2)
print(xored2_5)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 10, 0, 0, 0, 0, 0]


In [ ]:
print(xored1)
print(xored2)
print(xored2_5)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(xored1)
print(xored2)
print(xored2_5)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 12, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0]


In [ ]:
print(xored1)
print(xored2)
print(xored2_5)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 12, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0]


In [ ]:
print(xored1)
print(xored2)
print(xored2_5)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0]


In [ ]:
message = "28472B4B6250655368566D5971337436"
bitmg = bin(int(message, 16))
print(bitmg)
encrypted = encrypt(bitmg, bitkey)
print(encrypted)
print(hex(int(encrypted, 2))[2:].upper())

0b101000010001110010101101001011011000100101000001100101010100110110100001010110011011010101100101110001001100110111010000110110
10110110011101001000001111110110110101110101011011001101001011101010100011010011001110110011111001010101101000111100111010110110
B67483F6D756CD2EA8D33B3E55A3CEB6


In [ ]:
def bitor(x1, x2):
  ored = x1 or x2
  if ored:
    return 1
  else:
    return 0

In [ ]:
def bitxor(x1, x2):
  if x1 == x2:
    return 0
  else:
    return 1

take 100 keys

for each key, take the generated 16 plaintexts with one all property and rest constnts, store the results of each encryption in 'results' i.e 32 128 bit messages.

make 3 lists xored1, xored2 and xored2_5 of length 128 each, storing the xor of ith bit of all 16 plaintext encryption operations.

make 3 lists ored1, ored2, and ored2_5 of length 128 each, storing the or of ith bit of of xors for each round's results.


In [ ]:
# 100 keys
for i in range(100):
    key = ""
    for j in range(255):
      key += str(random.randint(0,1))
    key = "1" + key
    keys.append(key)

In [ ]:
# 16 plaintexts
initial_message = "28472B4B625065536856FD5971337436"
messages = []
bit_messages = []

for i in range(16):
    new_msg = str(hex(i)[2:]) + str(initial_message[1:])
    messages.append(new_msg.upper())

for message in messages:
    message = bin(int(message, 16))[2:]
    if len(message) < 128:
      message = "0"*(128 - len(message)) + message
    bit_messages.append(message)

In [ ]:
ored1 = ["0"]*128
ored2 = ["0"]*128
ored2_5 = ["0"]*128
for key in keys:
  xored1 = ["0"]*128
  xored2 = ["0"]*128
  xored2_5 = ["0"]*128
  for plaintext in bit_messages:
    results = round_encrypt(plaintext, key)
    round_result1 = results[0][0] 
    round_result2 = results[0][1]
    half_result = result[2]
    for i in range(len(round_result1)):
      xored1[i] = bitxor(round_result1[i], xored1[i])
      xored2[i] = bitxor(round_result2[i], xored2[i])
      xored2_5[i] = bitxor(half_result[i], xored2_5[i])
  for j in range(128):
    ored1[j] = bitor(xored1[j], ored1[j])
    ored2[j] = bitor(xored2[j], ored2[j])
    ored2_5[j] = bitor(xored2_5[j], ored2_5[j])

print(ored1)
print(ored2) 
print(ored2_5)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1

As can be seen, the or of xors of results of the first 2 rounds are all 0, meaning all bits maintain Balanced property until after the second round.
After that, till the 2.5 rounds completion, some bits don't maintain the property.